## Introduction to `robotoc` 
# 2: Robot Models

## The goal of this chapter
Here, we learn how to create robot models (`robotoc::Robot`) from URDF packages.  
Note that, `robotoc::Robot` is just a wrapper of [`Pinocchio`](https://github.com/stack-of-tasks/pinocchio)'s API.  
So, if you want to dive in deeper, please check [`Pinocchio`'s documentation](https://stack-of-tasks.github.io/pinocchio/).

## What is URDF?
URDF (universal robot description format) describes the robot.  
We need very little knowledge on URDF as long as we use off-the-shelf URDF files with `robotoc`.  
So don't worry even if you do not know well about URDF!  
You can see off-the-shelf URDF packages in this repository's `urdf` folder.    
As we can see there, a URDF package is composed of urdf file (`~.urdf`) and mesh files (e.g., `~.stl`, `~.dae`, or `~.obj`)   
The urdf file describes the tree structure of the multi-link robot and mesh files contain visual and collision information.   
If you'd like to know more, please see, e.g., the ROS tutorials http://wiki.ros.org/urdf/Tutorials.  

## Simplest example: a robot manipulator `iiwa14`

First, we construct `robotoc.Robot` (`robotoc::Robot` in C++) with a simple example.  
Note that C++'s `robotoc::Robot` has more rich methods than the following Python counterpart.

In [ ]:
import robotoc
model_info = robotoc.RobotModelInfo()
model_info.urdf_path = "urdf/iiwa_description/urdf/iiwa14.urdf"
iiwa14 = robotoc.Robot(model_info)

We can see the information of the robot as 

In [ ]:
print(iiwa14)

The printed information is as follows:
- `dimq`: the dimension of the configuration
- `dimv`: the dimension of the generalized velocity (not always the same as `dimq`)
- `dimu`: the dimension of the input torque
- `dim_passive`: the dimension of the passive joints including the floating base
- `frames`: frames of the robot. you can use to design the cost function and constraints on the end-effector.
- `joints`: each joint info.
- `effort limit`: limits of joint torques
- `velocity limit`: limits of joint velocities
- `lower position limit`, `upper position limit`: limits of joint positions

You can individually get some of them as

In [ ]:
iiwa14.dimq()

In [ ]:
iiwa14.dimv()

In [ ]:
iiwa14.dimu()

In [ ]:
iiwa14.dim_passive()

In [ ]:
iiwa14.joint_effort_limit()

In [ ]:
iiwa14.joint_velocity_limit()

In [ ]:
iiwa14.lower_joint_position_limit()

In [ ]:
iiwa14.upper_joint_position_limit()

Next, let's visualize them with random joint veocities.

In [ ]:
import numpy as np
steps = 100
dt = 0.1
q = []
q.append(iiwa14.generate_feasible_configuration())
for i in range(steps):
    v = np.random.randn(iiwa14.dimv())
    q.append(iiwa14.integrate_configuration(q[-1], v, dt))

time_discretization = robotoc.TimeDiscretization(T=dt*steps, N=steps)
    
viewer = robotoc.utils.TrajectoryViewer(model_info=model_info, viewer_type='meshcat')
viewer.display(time_discretization, q)

You can also call simple forward kinematics with the robot object.

In [ ]:
end_effector_frame = 'iiwa_link_ee_kuka'
for e in q:
    iiwa14.forward_kinematics(e)
    print('q: ', e)
    print('frame position: ', iiwa14.frame_position(end_effector_frame))
    print('frame rotation: ', iiwa14.frame_rotation(end_effector_frame))
    print('frame placement (SE3): ', iiwa14.frame_placement(end_effector_frame))

## Quadrupedal robot `ANYmal`

Second, we try a quadruped robot example.  
The quadrupedal robot typically involves **four point contacts**.  So we specify the names of the contact frames.  
We also specify the time-step parameter (or gain parameters) of the Baumgarte's stabilization method for acceleration-level rigid contact constraints (detail is found, e.g., in [this paper](https://www.researchgate.net/publication/234610391_A_Parametric_Study_on_the_Baumgarte_Stabilization_Method_for_Forward_Dynamics_of_Constrained_Multibody_Systems)).  
The appropriate parameter depends on the time step of MPC formulation.  So some numerical experimentation is necessary.   
In my experience, 2~10 times of time step of MPC work well.
Here, we set it as 0.04.
These information of each contact is summarized in `robotoc.ContactModelInfo` (`robotoc::ContactModelInfo` in C++) class, e.g..

In [ ]:
baumgarte_time_step = 0.04
robotoc.ContactModelInfo('FL_foot', baumgarte_time_step)

The robot has a **Floating base**. 
The model info such as the **URDF path**, **contact info**, and the **base type (floating base or fixed base)** are specified in `robotoc.RobotModelInfo` (`robotoc::RobotModelInfo` in C++), which represents the settings of a robot model.

In [ ]:
model_info = robotoc.RobotModelInfo()
model_info.urdf_path = 'urdf/anymal_b_simple_description/urdf/anymal.urdf'
model_info.base_joint_type = robotoc.BaseJointType.FloatingBase
baumgarte_time_step = 0.04
model_info.point_contacts = [robotoc.ContactModelInfo('LF_FOOT', baumgarte_time_step),
                             robotoc.ContactModelInfo('LH_FOOT', baumgarte_time_step),
                             robotoc.ContactModelInfo('RF_FOOT', baumgarte_time_step),
                             robotoc.ContactModelInfo('RH_FOOT', baumgarte_time_step)]
anymal = robotoc.Robot(model_info)

We can see the detail of the robot model as 

In [ ]:
print(anymal)

We can also display the robot.  

In [ ]:
steps = 100
dt = 0.1
q = []
q.append(anymal.generate_feasible_configuration())
q[0][0:3] = np.zeros(3)
for i in range(steps):
    v = np.concatenate([0.1*np.random.randn(6), np.random.randn(anymal.dimu())])
    q.append(anymal.integrate_configuration(q[-1], v, dt))

time_discretization = robotoc.TimeDiscretization(T=dt*steps, N=steps)

viewer = robotoc.utils.TrajectoryViewer(model_info=model_info, viewer_type='meshcat')
viewer.display(time_discretization, q)

## Humanoid robot `iCub`

Third, we try a humanoid robot example.  
Humanoid robots typically involve **two surface contacts**.  
We construct the robot model as the quadrupedal case as

In [ ]:
model_info = robotoc.RobotModelInfo()
model_info.urdf_path = 'urdf/icub_description/urdf/icub.urdf'
model_info.base_joint_type = robotoc.BaseJointType.FloatingBase
baumgarte_time_step = 0.05
model_info.surface_contacts = [robotoc.ContactModelInfo('l_sole', baumgarte_time_step),
                               robotoc.ContactModelInfo('r_sole', baumgarte_time_step)]
icub = robotoc.Robot(model_info)

We can see the detail of the robot model as 

In [ ]:
print(icub)

and display the robot

In [ ]:
steps = 100
dt = 0.1
q = []
q.append(icub.generate_feasible_configuration())
q[0][0:3] = np.zeros(3)
for i in range(steps):
    v = np.concatenate([0.1*np.random.randn(6), np.random.randn(icub.dimu())])
    q.append(icub.integrate_configuration(q[-1], v, dt))

time_discretization = robotoc.TimeDiscretization(T=dt*steps, N=steps)
    
viewer = robotoc.utils.TrajectoryViewer(model_info=model_info, viewer_type='meshcat')
viewer.display(time_discretization, q)